In [2]:
import numpy as np
from ipywidgets import interact
from matplotlib import pyplot as plt

dataset = "fxmovie"
scan_id = "c0001139"
slices = [167] #np.linspace(0, 167, num=17, dtype=int)
base_path = "/home/fvdenbergh/segmentation/data"

In [3]:
def load_aim(path_to_aim, read_log=True):
    reader = vtkbone.vtkboneAIMReader()
    reader.DataOnCellsOff()
    reader.SetFileName(path_to_aim)
    reader.Update()

    vtk_data = reader.GetOutput().GetPointData().GetScalars()
    data = vtk_to_numpy(vtk_data).reshape(reader.GetOutput().GetDimensions(), order='F')

    if read_log:
        m, b = get_aim_density_equation(reader.GetProcessingLog())
        return data, m, b
    else:
        return data, 0, 0


def rescale(data, m, b):
    return m * data + b

In [4]:
if dataset == "fxmovie":
    scan, m, b = load_aim(f"{base_path}/{dataset}/batch1/{scan_id}_crop.aim")
    endo_emb = np.load(f"{base_path}/{dataset}/batch1/embedding/{scan_id}_crop_endo_embedding.npy")
    peri_emb = np.load(f"{base_path}/{dataset}/batch1/embedding/{scan_id}_crop_peri_embedding.npy")
elif dataset == "skedoi":
    scan, m, b = load_aim(f"{base_path}/{dataset}/{scan_id}_crop.aim")
    endo_emb = np.load(f"{base_path}/{dataset}/embedding/{scan_id}_crop_endo_embedding.npy")
    peri_emb = np.load(f"{base_path}/{dataset}/embedding/{scan_id}_crop_peri_embedding.npy")
else:
    scan, m, b = None, 0, 0
    endo_emb, peri_emb = 0, 0
scan = rescale(scan, m, b)

endo_emb_norm = (endo_emb - endo_emb.min()) / (endo_emb.max() - endo_emb.min())
peri_emb_norm = (peri_emb - peri_emb.min()) / (peri_emb.max() - peri_emb.min())

NameError: name 'vtkbone' is not defined

In [3]:
def threshold_masks(peri_emb_, endo_emb_, peri_thresh, endo_thresh):
    cort_mask = (peri_emb_ < peri_thresh) * (endo_emb_ > endo_thresh)
    trab_mask = endo_emb_ < endo_thresh
    return cort_mask, trab_mask

# cort_mask = (phi_peri < 0) * (phi_endo > 0)
# trab_mask = phi_endo < 0
%matplotlib inline

def show_embeddings(peri_thresh=0, endo_thresh=0):
    for slice_idx in slices:
        f, ax = plt.subplots(2, 3, figsize=(12, 4))
        ax[0, 0].imshow(scan[..., slice_idx], cmap="gray")
        ax[0, 0].set_title(f"Original scan (slice {slice_idx + 1})")
        ax[0, 0].axis('off')
        ax[0, 1].imshow(endo_emb_norm[..., slice_idx], cmap="RdYlBu")
        ax[0, 1].set_title(f"Endosteal embedding (slice {slice_idx + 1})\n(neg (trab)=red, pos (cort)=blu)")
        ax[0, 1].axis('off')
        ax[0, 2].imshow(peri_emb_norm[..., slice_idx], cmap="RdYlBu")
        ax[0, 2].set_title(f"Periosteal embedding (slice {slice_idx + 1})\n(neg (cort)=red, pos (back)=blu)")
        ax[0, 2].axis('off')

        cort_mask, trab_mask = threshold_masks(peri_emb, endo_emb, peri_thresh, endo_thresh)

        ax[1, 0].imshow(scan[..., slice_idx], cmap="gray")
        ax[1, 0].imshow(trab_mask, cmap='Blues', alpha=0.7 * trab_mask)
        ax[1, 0].imshow(cort_mask, cmap='Greens', alpha=0.7 * cort_mask)
        ax[1, 0].set_title(f"Overlay (slice {slice_idx + 1})")
        ax[1, 0].axis('off')
        ax[1, 1].imshow(trab_mask, cmap="gray")
        ax[1, 1].set_title(f"Trab mask (slice {slice_idx + 1})")
        ax[1, 1].axis('off')
        ax[1, 2].imshow(cort_mask, cmap="gray")
        ax[1, 2].set_title(f"Cort mask (slice {slice_idx + 1})")
        ax[1, 2].axis('off')
        plt.show()


interact(show_embeddings, peri_thresh=(-5., 5., 0.01), cort_thresh=(-5., 5., 0.01))

interactive(children=(FloatSlider(value=0.0, description='peri_thresh', max=5.0, min=-5.0, step=0.01), IntSlid…

<function __main__.show_embeddings(peri_thresh=0, endo_thresh=0)>

In [5]:
!pip freeze

aiofiles @ file:///home/conda/feedstock_root/build_artifacts/aiofiles_1664378549280/work
aiohttp==3.8.6
aiosignal==1.3.1
aiosqlite @ file:///home/conda/feedstock_root/build_artifacts/aiosqlite_1715928379913/work
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1688651106312/work/dist
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1707233003401/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1649500563747/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1662382474514/work
async-timeout==4.0.3
asynctest==0.13.0
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1722977137225/work
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1702422572539/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file